In [1]:
%matplotlib inline
import re
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud, ImageColorGenerator


In [2]:
hm_data = pd.read_csv('../../HappyDB/happydb/data/cleaned_hm.csv')
hm_data.head()

,hmid,wid,reflection_period,original_hm,cleaned_hm,modified,num_sentence,ground_truth_category,predicted_category
0,27673,2053,24h,I went on a successful date with someone I fel...,I went on a successful date with someone I fel...,True,1,NaN,affection
1,27674,2,24h,I was happy when my son got 90% marks in his e...,I was happy when my son got 90% marks in his e...,True,1,NaN,affection
2,27675,1936,24h,I went to the gym this morning and did yoga.,I went to the gym this morning and did yoga.,True,1,NaN,exercise
3,27676,206,24h,We had a serious talk with some friends of our...,We had a serious talk with some friends of our...,True,2,bonding,bonding
4,27677,6227,24h,I went with grandchildren to butterfly display...,I went with grandchildren to butterfly display...,True,1,NaN,affection


In [12]:
hm_data["reflection_period"]=='3m'

0         False
1         False
2         False
3         False
4         False
5         False
6         False
7         False
8         False
9         False
10        False
11        False
12        False
13        False
14        False
15        False
16        False
17        False
18        False
19        False
20        False
21        False
22        False
23        False
24        False
25        False
26        False
27        False
28        False
29        False
          ...  
100505    False
100506    False
100507    False
100508    False
100509    False
100510    False
100511    False
100512    False
100513    False
100514    False
100515    False
100516    False
100517    False
100518    False
100519    False
100520    False
100521    False
100522    False
100523    False
100524    False
100525    False
100526    False
100527    False
100528    False
100529    False
100530    False
100531    False
100532    False
100533    False
100534    False
Name: reflection_period,

In [18]:
conditions = (hm_data["ground_truth_category"] == 'affection') | (hm_data["ground_truth_category"] == 'bonding')
conditions = (hm_data["reflection_period"]=='3m') & conditions
hm_3m = hm_data.loc[conditions]
hm_3m.head()

,hmid,wid,reflection_period,original_hm,cleaned_hm,modified,num_sentence,ground_truth_category,predicted_category
30513,58356,4863,3m,I spent the day with my wife playing glow in t...,I spent the day with my wife playing glow in t...,True,1,affection,affection
30537,58380,995,3m,My mom moving out of my house,My mom moving out of my house,True,1,affection,affection
30539,58382,5299,3m,I visited my aunt and uncle in Arkansas this w...,I visited my aunt and uncle in Arkansas this w...,True,2,affection,affection
30540,58383,562,3m,When I took my dog to the vet last week and sh...,When I took my dog to the vet last week and sh...,True,2,affection,affection
30546,58389,126,3m,I took my daughter to the beach for the first ...,I took my daughter to the beach for the first ...,True,1,affection,affection


In [20]:
hm_3m

,hmid,wid,reflection_period,original_hm,cleaned_hm,modified,num_sentence,ground_truth_category,predicted_category
30513,58356,4863,3m,I spent the day with my wife playing glow in t...,I spent the day with my wife playing glow in t...,True,1,affection,affection
30537,58380,995,3m,My mom moving out of my house,My mom moving out of my house,True,1,affection,affection
30539,58382,5299,3m,I visited my aunt and uncle in Arkansas this w...,I visited my aunt and uncle in Arkansas this w...,True,2,affection,affection
30540,58383,562,3m,When I took my dog to the vet last week and sh...,When I took my dog to the vet last week and sh...,True,2,affection,affection
30546,58389,126,3m,I took my daughter to the beach for the first ...,I took my daughter to the beach for the first ...,True,1,affection,affection
30557,58401,1426,3m,My oldest daughter was competing in the Region...,My oldest daughter was competing in the Region...,True,2,affection,affection
30586,58430,312,3m,Received a notice from the ACT that my younges...,Received a notice from the ACT that my younges...,True,1,affection,affection
30591,58435,2641,3m,I met a girl and ended up going on a date with...,I met a girl and ended up going on a date with...,True,1,affection,affection
30604,58448,2295,3m,I visited my daughter & granddaughters in New ...,I visited my daughter & granddaughters in New ...,True,1,affection,affection
30617,58461,2915,3m,We went on vacation last week and took our gra...,We went on vacation last week and took our gra...,True,1,affection,affection
